task name: Implement One-Class Matrix Factorization

task description: Implement the provided C# code for One-Class Matrix Factorization using ML.NET

In [ ]:
#r "nuget:Microsoft.ML,1.6.0"
#r "nuget:Microsoft.ML.Recommender,0.21.0-preview.23266.6"

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
using System.Collections.Generic;

// The following variables defines the shape of a matrix. Its shape is 
// _synthesizedMatrixRowCount-by-_synthesizedMatrixColumnCount.
// Because in ML.NET key type's minimal value is zero, the first row index
// is always zero in C# data structure (e.g., MatrixColumnIndex=0 and 
// MatrixRowIndex=0 in MatrixElement below specifies the value at the
// upper-left corner in the training matrix). If user's row index
// starts with 1, their row index 1 would be mapped to the 2nd row in matrix
// factorization module and their first row may contain no values.
// This behavior is also true to column index.
private const uint _synthesizedMatrixColumnCount = 60;
private const uint _synthesizedMatrixRowCount = 100;

public class MatrixElement
{
    // Matrix column index. Its allowed range is from 0 to _synthesizedMatrixColumnCount - 1.
    [KeyType(_synthesizedMatrixColumnCount)]
    public uint MatrixColumnIndex { get; set; }
    // Matrix row index. Its allowed range is from 0 to _synthesizedMatrixRowCount - 1.
    [KeyType(_synthesizedMatrixRowCount)]
    public uint MatrixRowIndex { get; set; }
    // The value at the MatrixColumnIndex-th column and the MatrixRowIndex-th row.
    public float Value { get; set; }
    // The predicted value at the MatrixColumnIndex-th column and the MatrixRowIndex-th row.
    public float Score { get; set; }
}

// Create an in-memory matrix as a list of tuples (column index, row index, value). Notice that one-class matrix factorization handle scenerios where only positive signals (e.g., on Facebook, only likes are recorded and no dislike before) can be observed so that all values are set to 1.
private static void GetOneClassMatrix(out List<MatrixElement> observedMatrix, out List<MatrixElement> fullMatrix)
{
    // The matrix factorization model will be trained only using observedMatrix but we will see it can learn all information carried sin fullMatrix.
    observedMatrix = new List<MatrixElement>();
    fullMatrix = new List<MatrixElement>();
    for (uint i = 0; i < _synthesizedMatrixColumnCount; ++i)
        for (uint j = 0; j < _synthesizedMatrixRowCount; ++j)
        {
            if ((i + j) % 10 == 0)
            {
                // Set observed elements' values to 1 (means like).
                observedMatrix.Add(new MatrixElement()
                {
                    MatrixColumnIndex = i,
                    MatrixRowIndex = j,
                    Value = 1,
                    Score = 0
                });
                fullMatrix.Add(new MatrixElement()
                {
                    MatrixColumnIndex = i,
                    MatrixRowIndex = j,
                    Value = 1,
                    Score = 0
                });
            }
            else
                // Set unobserved elements' values to 0.15, a value smaller than observed values (means dislike).
                fullMatrix.Add(new MatrixElement()
                {
                    MatrixColumnIndex = i,
                    MatrixRowIndex = j,
                    Value = 0.15f,
                    Score = 0
                });
        }
}

GetOneClassMatrix(out List<MatrixElement> data, out List<MatrixElement> testData);
var dataView = mlContext.Data.LoadFromEnumerable(data);

In [ ]:
var options = new MatrixFactorizationTrainer.Options
{
    MatrixColumnIndexColumnName = nameof(MatrixElement.MatrixColumnIndex),
    MatrixRowIndexColumnName = nameof(MatrixElement.MatrixRowIndex),
    LabelColumnName = nameof(MatrixElement.Value),
    NumberOfIterations = 20,
    NumberOfThreads = 8,
    ApproximationRank = 32,
    Alpha = 1,
    C = 0.15,
    LossFunction = MatrixFactorizationTrainer.LossFunctionType.SquareLossOneClass
};

var pipeline = mlContext.Recommendation().Trainers.MatrixFactorization(options);

In [ ]:
var model = pipeline.Fit(dataView);

In [ ]:
var prediction = model.Transform(mlContext.Data.LoadFromEnumerable(testData));

In [ ]:
var results = mlContext.Data.CreateEnumerable<MatrixElement>(prediction, false).ToList();
foreach (var pred in results.Take(15))
{
    Console.WriteLine($"Predicted value at row {pred.MatrixRowIndex - 1} and column {pred.MatrixColumnIndex - 1} is {pred.Score} and its expected value is {pred.Value}.");
}